In [ ]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds
from tools import EarlyStopping

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# In[2]:
torch.set_num_threads(8)
torch.get_num_threads()

In [ ]:
class SemiDataset(Dataset):
    def __init__(self, df, fea_cols, y_cols):        
        self.X = df[fea_cols].values
        self.y = df[y_cols].values
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
    

df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)

print(df_train.shape, df_test.shape)

layer_cols = [c for c in df_train.columns if 'layer_' in c]
fea_cols = [c for c in df_train.columns if c not in layer_cols]

len(fea_cols), len(layer_cols)

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(df_train[fea_cols].values)

scaler = joblib.load('scaler.bin')

df_train[fea_cols] = scaler.transform(df_train[fea_cols].values)
df_test[fea_cols] = scaler.transform(df_test[fea_cols].values)

torch.manual_seed(81511991154)
torch.initial_seed()

df_model = df_train

dataset = SemiDataset(df_model[fea_cols + layer_cols], fea_cols, layer_cols)

train_set, val_set = torch.utils.data.random_split(dataset, [700000, 110000])

print(len(train_set), len(val_set))

batch_size = 25000
num_workers = 4

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=num_workers)
val_loader = DataLoader(dataset=val_set, batch_size=22000)

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

#### Model

In [ ]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.device = device
        self.model = model#.to(self.device)
        self.criterion = criterion#.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

        print(self.model.train())
        pass
    
    def set_optimizer(self, optimizer):
        self.optimizer = optimizer
        
    def set_scheduler(self, scheduler):
        self.scheduler = scheduler
        
    def train_(self, X_batch, y_batch):
        self.model.train()
        total_loss = 0
        X_batch = X_batch.to(self.device)
        y_batch = y_batch.to(self.device)

        y_pred = self.model(X_batch)
#             print(y_pred, y_batch)

        loss = self.criterion(y_pred, y_batch)
        total_loss = total_loss + loss.item()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.scheduler.step()
        
        return total_loss / len(data_loader)
    
    def train(self, data_loader):
        self.model.train()
        total_loss = 0
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            y_pred = self.model(X_batch)
#             print(y_pred, y_batch)
            
            loss = self.criterion(y_pred, y_batch)
            total_loss = total_loss + loss.item()

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        self.scheduler.step()
        
        return total_loss / len(data_loader)
    
    def eval(self, data_loader):
        self.model.eval()
        total_loss = 0
#         print('valid_loader', len(valid_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            with torch.no_grad():
                y_pred = self.model(X_batch)
                loss = self.criterion(y_pred, y_batch)
                total_loss = total_loss + loss.item()
        return total_loss / len(data_loader)

    def save(self, model_path='checkpoint.pt'):
        joblib.dump(self.model, model_path)
        return
    
    def load(self, model_path='checkpoint.pt'):
        self.model = joblib.load(model_path)
        return

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
    def forward(self, x):
        return self.model(x)
 

In [ ]:
# models = [
#     'model/20200125T200128_0.4469637870788574.model',
#     'model/20200124T233212_0.4282313088575999.model',
#     'model/20200125T200128_0.4469637870788574.model',
#     'model/20200129T232141_0.4588648915290833.model',
#     'model/20200130T082430_0.4571056246757507.model',
#     'model/20200130T085842_0.47272029519081116.model',
#     'model/20200130T140408_0.4705016195774078.model',
#     'model/20200125T200128_0.3261864085992177.model',    
#     'model/20200130T140408_0.3449194014072418.model',
#     'model/20200131T111430_0.4638769030570984.model',
# ]

# train_preds_list = []
# valid_preds_list = []
# test_preds_list = []
# for i, model_path in enumerate(models):
#     model = joblib.load(model_path)
#     print(model_path)
# #     print(model)
#     criterion = nn.L1Loss(reduction='mean').to(device)
    
#     model.train()
#     total_loss = 0
#     preds = []
#     train_y_true = []
#     for data in train_loader:
#         X_batch, y_batch = data
#         X_batch = X_batch.to(device)
#         y_batch = y_batch.to(device)
#         train_y_true.extend(y_batch.cpu().numpy())

#         with torch.no_grad():
#             y_pred = model(X_batch)
#             preds.extend(y_pred.cpu().numpy())
#             loss = criterion(y_pred, y_batch)
#             total_loss = total_loss + loss.item()

#     train_loss_min = total_loss / len(train_loader)
#     print('train_loss_min', train_loss_min)
#     preds = np.stack(preds)
#     train_preds_list.append(preds)
    
#     model.eval()
#     total_loss = 0
#     preds = []
#     valid_y_true = []
#     for data in val_loader:
#         X_batch, y_batch = data
#         X_batch = X_batch.to(device)
#         y_batch = y_batch.to(device)
#         valid_y_true.extend(y_batch.cpu().numpy())

#         with torch.no_grad():
#             y_pred = model(X_batch)
#             preds.extend(y_pred.cpu().numpy())
#             loss = criterion(y_pred, y_batch)
#             total_loss = total_loss + loss.item()

#     val_loss_min = total_loss / len(val_loader)
#     print('val_loss_min', val_loss_min)
#     preds = np.stack(preds)
#     valid_preds_list.append(preds)
    
    
#     model.eval()
#     y_pred = model(torch.Tensor(df_test[fea_cols].values).to(device))    
#     y_pred = np.stack(y_pred.cpu().detach().numpy())
#     test_preds_list.append(y_pred)


In [ ]:
# df_train = pd.DataFrame(np.concatenate(train_preds_list, axis=1))

# cols = []
# for i in range(len(models)):
#     cols.extend([f'pred_{i}_1', f'pred_{i}_2', f'pred_{i}_3', f'pred_{i}_4'])

# df_train.columns = cols
# df_train_y = pd.DataFrame(np.stack(train_y_true), columns = ['layer_1', 'layer_2', 'layer_3', 'layer_4'])

# df_train = pd.concat([df_train, df_train_y], axis=1)

# df_train

In [ ]:
# df_valid = pd.DataFrame(np.concatenate(valid_preds_list, axis=1))

# cols = []
# for i in range(len(models)):
#     cols.extend([f'pred_{i}_1', f'pred_{i}_2', f'pred_{i}_3', f'pred_{i}_4'])

# df_valid.columns = cols
# df_valid_y = pd.DataFrame(np.stack(valid_y_true), columns = ['layer_1', 'layer_2', 'layer_3', 'layer_4'])

# df_valid = pd.concat([df_valid, df_valid_y], axis=1)

# df_valid

In [ ]:
# df_score = pd.DataFrame(np.concatenate(test_preds_list, axis=1))

# cols = []
# for i in range(len(models)):
#     cols.extend([f'pred_{i}_1', f'pred_{i}_2', f'pred_{i}_3', f'pred_{i}_4'])

# df_score.columns = cols

# df_score

In [ ]:
df_train = joblib.load('df_pred_train_10.pkl')
df_valid = joblib.load('df_pred_valid_10.pkl')
df_score = joblib.load('df_pred_score_10.pkl')

In [ ]:
# joblib.dump(df_train, 'df_pred_train_10.pkl')
# joblib.dump(df_valid, 'df_pred_valid_10.pkl')
# joblib.dump(df_score, 'df_pred_score_10.pkl')

#### LR

In [ ]:
# fea_cols

# from sklearn.linear_model import Ridge

# clf = Ridge(alpha=1.0)
# clf.fit(df_train[fea_cols], df_train['layer_2'])

# clf.predict(df_valid[fea_cols])

# mean_absolute_error(df_valid['layer_2'], clf.predict(df_valid[fea_cols]))

# from sklearn.linear_model import LinearRegression

# reg = LinearRegression().fit(df_train[fea_cols], df_train['layer_2'])
# reg.score(df_train[fea_cols], df_train['layer_2'])

# reg.coef_

# reg.intercept_

# reg.predict(df_valid[fea_cols])

# df_valid['layer_2']

# from sklearn.metrics import mean_absolute_error

# mean_absolute_error(df_valid['layer_2'], reg.predict(df_valid[fea_cols]))

# # y_true = [[0.5, 1], [-1, 1], [7, -6]]
# # y_pred = [[0, 2], [-1, 2], [8, -5]]
# # mean_absolute_error(y_true, y_pred)

# # mean_absolute_error(y_true, y_pred, multioutput='raw_values')

# # mean_absolute_error(y_true, y_pred, multioutput=[0.3, 0.7])

#### LGB

In [ ]:
# for i in range(1, 5):
#     layer_cols = [f'layer_{i}']
#     fea_cols = [c for c in df_train.columns if c not in layer_cols and c[-1] == str(i)]

#     print(fea_cols)
#     print(layer_cols)

#     model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
#     print('model_ts', model_ts)

#     initscore_filename = ''
#     params = {
#         'boosting':'gbdt',
#         'num_leaves': 7,
#     #     'max_depth': 3,
#         'objective': 'l2',
#         'metric':'l1',
#         'num_threads': 14,
#         'learning_rate': 0.01,
#         'device_type':'gpu',

#     }
#     print(params)

#     data_params = {
#     #     'max_bin':127,
#     #     'enable_bundle': False,
#     }
#     print(data_params)

#     num_round = 50000
#     print('num_round:', num_round)

#     train_set = lgb.Dataset(df_train[fea_cols].values, label=df_train[layer_cols[0]].values, params=data_params)
#     val_set = lgb.Dataset(df_valid[fea_cols].values, label=df_valid[layer_cols[0]].values, params=data_params)

#     evals_result = {}
#     model = lgb.train(params, train_set, num_round, early_stopping_rounds=500, 
#                             valid_sets=[train_set, val_set],
#                             verbose_eval=1000,
#                             evals_result=evals_result,
#                            )
#     df_score[layer_cols[0]] = model.predict(df_score[fea_cols].values)
#     print(df_score[fea_cols + layer_cols])


In [ ]:
# df_score.index.name = 'id'
# df_score[[c for c in df_score if 'layer' in c]]

# for i in range(1,5):
#     df_score[f'layer_{i}'] = df_score[[c for c in df_score.columns if c[0] == 'p' and c[-1] == str(i)]].mean(axis=1)

# df_score

# df_score[[c for c in df_score if 'layer' in c]]

# df_score[[c for c in df_score if 'layer' in c]].to_csv('eeeee1.csv')

# model.predict(df_score[fea_cols].values)

# df_valid['layer_1']


#### DNN

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.m1 = torch.nn.Sequential(
            torch.nn.Linear(input_size, 128), torch.nn.BatchNorm1d(128), act, dropout, 
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout, 
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 1)
        )
        self.m2 = torch.nn.Sequential(
            torch.nn.Linear(input_size, 128), torch.nn.BatchNorm1d(128), act, dropout, 
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout, 
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 1)
        )
        self.m3 = torch.nn.Sequential(
            torch.nn.Linear(input_size, 128), torch.nn.BatchNorm1d(128), act, dropout, 
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout, 
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 1)
        )
        self.m4 = torch.nn.Sequential(
            torch.nn.Linear(input_size, 128), torch.nn.BatchNorm1d(128), act, dropout, 
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout, 
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(128, 1)
        )
    def forward(self, x):
        x1 = x[:,0::4]
        x2 = x[:,1::4]
        x3 = x[:,2::4]
        x4 = x[:,3::4]
        return torch.cat([self.m1(x1), self.m2(x2), self.m3(x3), self.m4(x4)], axis=1)
 

In [ ]:
pred_cols = [c for c in df_train.columns if 'pred' in c]

pred_cols

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(pd.concat([df_train, df_valid])[pred_cols].values)

# df_train[pred_cols] = scaler.transform(df_train[pred_cols].values)
# df_valid[pred_cols] = scaler.transform(df_valid[pred_cols].values)
# df_score[pred_cols] = scaler.transform(df_score[pred_cols].values)

In [ ]:
df_train

In [ ]:
# layer_cols = [c for c in df_train.columns if 'layer' in c]
# fea_cols = pred_cols
# print(fea_cols)
# print(layer_cols)

# train_set = SemiDataset(df_train, fea_cols, layer_cols)
# val_set = SemiDataset(df_valid, fea_cols, layer_cols)

# print(len(train_set), len(val_set))

# #     batch_size = 700000 // 2
# batch_size = 25000
# num_workers = 8

# train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
# val_loader = DataLoader(dataset=val_set, batch_size=110000)

# print(f'batch_size {batch_size} num_workers {num_workers}')
# print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

# model = DNNModel(input_size=len(fea_cols) // 4, dropout_probability=0).to(device)

# criterion = nn.L1Loss(reduction='mean').to(device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)
# scheduler = StepLR(optimizer, step_size=50, gamma=1.0)
# trainer = Trainer(model, criterion, optimizer, scheduler, device)


# val_loss_min = np.Inf

# lr_list = [
#     (0.01, 20),
#     (0.001, 20),
#     (0.0001, 20),
#     (0.00003, 20),
#     (0.00001, 20),
#     (0.000005, 20),
# ]


# total_epoch = 10000

# for lr, patience in lr_list:
#     print(lr, patience)
#     if os.path.isfile('stop.flag'):
#         print('stop!')
#         break

#     early_stopping = EarlyStopping(patience=patience, min_epoch=1, verbose=True)
#     early_stopping.val_loss_min = val_loss_min
#     early_stopping.best_score = None if val_loss_min==np.Inf else -val_loss_min 

# #     criterion = nn.L1Loss(reduction='mean').to(device)

#     trainer.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# #     trainer.optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# #     trainer.optimizer = torch.optim.SGD(model.parameters(), lr=lr)

#     trainer.scheduler = StepLR(trainer.optimizer, step_size=50, gamma=1.0)

#     for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
#         if os.path.isfile('stop.flag'):
#             print(f'{e} stop!')
#             break

#         train_loss = trainer.train(train_loader)

#         if e % 1 == 0:
#             valid_loss = trainer.eval(val_loader)
#     #         valid_loss = train_loss

#             ts = datetime.now().strftime('%Y%m%dT%H%M%S')
#             print(f'[{ts}] Epock {e} / {total_epoch}\t lr {trainer.scheduler.get_lr()[0]}')
#             print(f'  train_loss: {train_loss}  valid_loss: {valid_loss}')

#             early_stopping(valid_loss, model)

#             if early_stopping.early_stop:
#                 print("\tEarly stopping epoch {}, valid loss {}".format(e, early_stopping.val_loss_min))
#                 break

#     model.load_state_dict(torch.load('model/checkpoint.pt'))
#     val_loss_min = early_stopping.val_loss_min


#     model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
#     print(model_ts)
#     model_path = 'model/e{}_{}'.format(model_ts, val_loss_min)
#     trainer.save('{}.model'.format(model_path))
#     print(model_path)




In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3, dims=32):
        super(DNNModel,self).__init__()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.m1 = torch.nn.Sequential(
            torch.nn.Linear(input_size, dims), torch.nn.BatchNorm1d(dims), act, dropout, 
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout, 
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
#             torch.nn.Linear(128, 256), torch.nn.BatchNorm1d(256), act, dropout,
#             torch.nn.Linear(256, 256), torch.nn.BatchNorm1d(256), act, dropout,
#             torch.nn.Linear(256, 256), torch.nn.BatchNorm1d(256), act, dropout,
#             torch.nn.Linear(256, 128), torch.nn.BatchNorm1d(128), act, dropout,
#             torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
#             torch.nn.Linear(128, 128), torch.nn.BatchNorm1d(128), act, dropout,
            torch.nn.Linear(dims, 1)
        )
    def forward(self, x):
        return self.m1(x)
 

In [ ]:
# X_batch = []
# y_batch = []
# for i in [1, 2, 3, 4]:
#     layer_cols = [f'layer_{i}']
#     fea_cols = [c for c in df_train.columns if c not in layer_cols and c[-1] == str(i)]
#     X_batch.append(torch.Tensor(df_train[fea_cols].values).to(device))
#     y_batch.append(torch.Tensor(df_train[layer_cols].values).to(device))


In [ ]:
for dims in range(192, 550, 32):
    print('dims', dims)
    model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
    for i in [1, 2, 3, 4]:
        layer_cols = [f'layer_{i}']
        fea_cols = [c for c in df_train.columns if c not in layer_cols and c[-1] == str(i)]
        print(fea_cols)
        print(layer_cols)

        train_set = SemiDataset(df_train, fea_cols, layer_cols)
        val_set = SemiDataset(df_valid, fea_cols, layer_cols)

        print(len(train_set), len(val_set))

#         batch_size = 700000 // 2
        batch_size = 70000
        num_workers = 8

        train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
        val_loader = DataLoader(dataset=val_set, batch_size=110000)

        print(f'batch_size {batch_size} num_workers {num_workers}')
        print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

        model = DNNModel(input_size=len(fea_cols), dropout_probability=0, dims=dims).to(device)

        criterion = nn.L1Loss(reduction='mean').to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)
        scheduler = StepLR(optimizer, step_size=50, gamma=1.0)
        trainer = Trainer(model, criterion, optimizer, scheduler, device)


        val_loss_min = np.Inf

        lr_list = [
            (0.01, 20),
            (0.001, 20),
            (0.0001, 20),
            (0.00003, 20),
            (0.00001, 20),
            (0.000005, 20),
        ]


        total_epoch = 10000

        for lr, patience in lr_list:
            print(lr, patience)
            if os.path.isfile('stop.flag'):
                print('stop!')
                break

            early_stopping = EarlyStopping(patience=patience, min_epoch=1, verbose=True)
            early_stopping.val_loss_min = val_loss_min
            early_stopping.best_score = None if val_loss_min==np.Inf else -val_loss_min 

        #     criterion = nn.L1Loss(reduction='mean').to(device)

            trainer.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        #     trainer.optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
        #     trainer.optimizer = torch.optim.SGD(model.parameters(), lr=lr)

            trainer.scheduler = StepLR(trainer.optimizer, step_size=50, gamma=1.0)

            for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
                if os.path.isfile('stop.flag'):
                    print(f'{e} stop!')
                    break

                train_loss = trainer.train(train_loader)
#                 train_loss = trainer.train_(X_batch[i], y_batch[i])

                if e % 1 == 0:
                    valid_loss = trainer.eval(val_loader)
            #         valid_loss = train_loss

                    ts = datetime.now().strftime('%Y%m%dT%H%M%S')
                    print(f'[{ts}] Epock {e} / {total_epoch}\t lr {trainer.scheduler.get_lr()[0]}')
                    print(f'  train_loss: {train_loss}  valid_loss: {valid_loss}')

                    early_stopping(valid_loss, model)

                    if early_stopping.early_stop:
                        print("\tEarly stopping epoch {}, valid loss {}".format(e, early_stopping.val_loss_min))
                        break

            model.load_state_dict(torch.load('model/checkpoint.pt'))
            val_loss_min = early_stopping.val_loss_min



            print(model_ts)
            model_path = 'model/{}_e{}_{}_{}'.format(model_ts, dims, i, val_loss_min)
            trainer.save('{}.model'.format(model_path))
            print(model_path)

    #     break



In [ ]:
# e_models =[
#     'model/20200131T225354_e1_0.32568883895874023.model',
#     'model/20200131T225354_e2_0.5253744721412659.model',
#     'model/20200131T225354_e3_0.3913254737854004.model',
#     'model/20200131T225354_e4_0.3752939999103546.model',
# ]

# for i in range(1, 5):
#     layer_cols = [f'layer_{i}']
#     fea_cols = [c for c in df_score.columns if c not in layer_cols and c[-1] == str(i)]
#     print(fea_cols)
#     print(layer_cols)
#     y_pred = joblib.load(e_models[i-1])(torch.Tensor(df_score[fea_cols].values).to(device))    
#     df_score[layer_cols[0]] = y_pred.cpu().detach().numpy()
#     print(df_score[layer_cols + fea_cols])



In [ ]:
# e_models = [
#     'model/e_1_20200131T202847_0.322307288646698.model',
# #     'model/e_1_20200131T221837_0.33561456203460693.model',
#     'model/e_2_20200131T213251_0.5193350911140442.model',
#     'model/e_3_20200131T214620_0.38977283239364624.model',
#     'model/e_4_20200131T211402_0.37712812423706055.model',
# #     'model/e_4_20200131T220102_0.3780273497104645.model',
# #     'model/e_3_20200131T214620_0.38977283239364624.model',
# #     'model/e_4_20200131T211402_0.37712812423706055.model',
# #     'model/e_3_20200131T205640_0.3976164758205414.model',
# # #     'model/e_2_20200131T204154_0.5304651260375977.model',
# #     'model/e_1_20200131T202847_0.322307288646698.model',
# ]
# for i in range(1, 5):
#     layer_cols = [f'layer_{i}']
#     fea_cols = [c for c in df_score.columns if c not in layer_cols and c[-1] == str(i)]
#     print(fea_cols)
#     print(layer_cols)
#     y_pred = joblib.load(e_models[i-1])(torch.Tensor(df_score[fea_cols].values).to(device))    
#     df_score[layer_cols[0]] = y_pred.cpu().detach().numpy()
#     print(df_score[layer_cols + fea_cols])

In [ ]:
df_score.index.name = 'id'
layer_cols = [c for c in df_score.columns if 'la' in c]
ts = datetime.now().strftime('%Y%m%dT%H%M%S')
df_score[layer_cols].to_csv(f'submit/{ts}_e.csv')

In [ ]:


# #### Pred Train

# criterion = nn.L1Loss(reduction='mean').to(device)
# model.train()
# total_loss = 0
# preds = []
# y_true = []
# for data in train_loader:
#     X_batch, y_batch = data
#     X_batch = X_batch.to(device)
#     y_batch = y_batch.to(device)
#     y_true.extend(y_batch)
    
#     with torch.no_grad():
#         y_pred = model(X_batch)
#         preds.extend(y_pred)
#         loss = criterion(y_pred, y_batch)
#         total_loss = total_loss + loss.item()
        
# train_loss_min = total_loss / len(train_loader)
# train_loss_min

# y_true

# preds

# #### Pred Valid

# criterion = nn.L1Loss(reduction='mean').to(device)
# model.eval()
# total_loss = 0
# for data in val_loader:
#     X_batch, y_batch = data
#     X_batch = X_batch.to(device)
#     y_batch = y_batch.to(device)
    
#     with torch.no_grad():
#         y_pred = model(X_batch)
#         loss = criterion(y_pred, y_batch)
#         total_loss = total_loss + loss.item()
        
# val_loss_min = total_loss / len(val_loader)
# val_loss_min

# #### Pred Test 

# model.eval()
# y_pred = model(torch.Tensor(df_test[fea_cols].values).to(device))    
# print(y_pred)

# ts = datetime.now().strftime('%Y%m%dT%H%M%S')

# df_submit = pd.read_csv('input/sample_submission.csv', index_col=0)

# df_submit[layer_cols] = y_pred.cpu().detach().numpy()
# df_submit.to_csv(f'submit/{ts}_{val_loss_min}.csv')

# print(ts, val_loss_min)

# # model.load_state_dict(torch.load('model/20200123T215856_0.6811393996079763.model'))

